In [1]:
import polars as pl
from polars import col as c
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp
from distflow_algorithm import DistFlow
import time 
import pandapower as pp
import pandapower.networks as pn
from polars_function import get_transfo_admittance, get_transfo_impedance, get_transfo_conductance, get_transfo_imaginary_component

from data_connector import pandapower_to_distflow

In [2]:

net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2

In [3]:
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 2.5711982250213623


In [13]:
s_base = 1e8
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0]**2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)


In [14]:
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

v0_sq = distflow.v_in_sq*v_ext_grid_sq
node_data = pl.DataFrame({"node_id": distflow.node_list}).join(node_data, on="node_id", how="left")
s_node = node_data["p_node_pu"].to_numpy() + 1j*node_data["q_node_pu"].to_numpy()


In [19]:
from more_itertools import locate
from itertools import product
from copy import deepcopy

v_in_sq: np.array = np.array([1.0]*len(distflow.node_list)) # type: ignore
dv_prop_np: np.array = deepcopy(distflow.h_np).transpose().astype(float) # type: ignore



In [21]:
dv_prop_np

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 0., 0., 1.]])

In [25]:
dv_prop_np * distflow.node_parameters["n_transfo"].to_list()

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.04286038, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.04286038, 1.09048374, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 1.04286038, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 1.04286038, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [1.        , 1.04286038, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [28]:
distflow.v_in_sq

array([1.        , 1.04286038, 1.13722229, ..., 1.13722229, 1.13722229,
       1.13722229])

In [27]:

for data in distflow.node_parameters.filter(c("n_transfo") != 1).to_dicts():
    print(data)
    upstream_node: list[int] = list(locate(
        distflow.node_list, lambda x: x in nx.ancestors(distflow.nx_tree_grid, data["node_id"])
    ))
    if len(upstream_node) != 0:
        # Find array index pairs for the voltage drop matrix
        downstream_node: list[int] = list(locate(
            distflow.node_list, 
            lambda x: x in [data["node_id"]] + list(nx.descendants(distflow.nx_tree_grid, data["node_id"]))
        ))
        rows, cols = zip(*list(product(downstream_node, upstream_node)))
        # Update the voltage drop matrix
        distflow.dv_prop_np[rows, cols] = data["n_transfo"]*distflow.dv_prop_np[rows, cols]
        distflow.v_in_sq[downstream_node] = data["n_transfo"]*distflow.v_in_sq[downstream_node]

{'node_id': '5377', 'n_transfo': 1.0428603814572701, 'r_pu': 0.012196422102230578, 'x_pu': 0.33582212229657793, 'g_tra_pu': 0.0002397580578697483, 'b_tra_pu': 0.006340028281962545, 'name': '', 'type': 'transformer'}
{'node_id': '5473', 'n_transfo': 1.0904837409298573, 'r_pu': 0.22932000000000002, 'x_pu': 5.287029065325817, 'g_tra_pu': 8.072562358276645e-06, 'b_tra_pu': 0.00022661363244841286, 'name': '', 'type': 'transformer'}
{'node_id': '3195', 'n_transfo': 1.0904837409298573, 'r_pu': 0.735, 'x_pu': 6.961305552839927, 'g_tra_pu': 5.895691609977325e-06, 'b_tra_pu': 0.0001141335415652908, 'name': '', 'type': 'transformer'}
{'node_id': '5355', 'n_transfo': 1.0904837409298573, 'r_pu': 0.7166250000000001, 'x_pu': 6.576068248533845, 'g_tra_pu': 7.07482993197279e-06, 'b_tra_pu': 0.0001812678840090239, 'name': '', 'type': 'transformer'}
{'node_id': '898', 'n_transfo': 1.0904837409298573, 'r_pu': 0.144703125, 'x_pu': 4.131841919318821, 'g_tra_pu': 9.160997732426304e-06, 'b_tra_pu': 0.00029010

In [6]:
tic = time.time()
p_flow, q_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq)
print("f:", time.time() - tic)

0.4415778676426103 4.935402478800775
5594
58
0.06952412077314762 0.10179519307759133
5594
1994
0.011944599038317794 0.040663811816012085
5594
2951
0.019604556104772408 0.09206465455213309
5594
3641
0.03238850527857062 0.07943959385099131
5594
3541
0.03307717571381241 0.019822177664975626
5594
2867
0.04159259399768267 0.04011405810791402
5594
2941
0.04170248053984227 0.07071327788737491
5594
3085
0.01392075339670873 0.09722154994266052
5594
3163
0.013688072511017557 0.09456601172511725
5594
3246
0.011379408729340623 0.09655506845551542
5594
3356
0.00823861942249371 0.08058216381956829
5594
3541
0.007891777350391926 0.06008039901826212
5594
3727
0.005192791406344988 0.04870470309766972
5594
3855
0.0047799803465278234 0.03800240576968328
5594
3975
0.005309187119936443 0.027331764298915928
5594
4133
0.004919862633570027 0.02397177023671926
5594
4204
0.0032958339305524387 0.0196941848072818
5594
4314
0.002890178496195883 0.015742947718371703
5594
4410
0.0023669806715513886 0.015650016208129

In [7]:
i.max()

2.4225601074205256

In [8]:
np.sqrt(3)*400*1400

969948.4522385712

In [9]:
p_flow.max()

2.3242162812764047

In [10]:
node_data.with_columns(
    pl.Series(i).alias("i_pu"),
).with_columns(
    (1e6/(c("v_base")*np.sqrt(3))*c("i_pu")).alias("i")
)

node_id,v_base,p_node_pu,q_node_pu,i_pu,i
str,f64,f64,f64,f64,f64
"""5593""",130000.0,0.0,0.0,2.42256,10.758967
"""5377""",18200.0,0.0,0.0,2.381034,75.532439
"""5473""",400.0,0.0,0.0,0.100156,144.56237
"""1453""",18200.0,0.0,0.0,0.033267,1.055302
"""5588""",18200.0,0.0,0.0,0.044133,1.400009
…,…,…,…,…,…
"""5477""",400.0,0.001091,0.0,0.001049,1.514611
"""669""",400.0,0.000172,0.0,0.000184,0.266124
"""1704""",400.0,0.000276,0.0,0.001511,2.18087


In [11]:
v.max()

1.3015981194970652

In [12]:
line_res_pp = pl.DataFrame({
    "node_id": list(net.line["to_bus"]),
    "p_pp": list(net.res_line["p_from_mw"]),
    "q_pp": list(net.res_line["q_from_mvar"])
})

result = node_data.select(
    c("node_id").cast(pl.Int32),
    pl.Series(v).alias("v"),
    pl.Series(p_flow).alias("p"),
    pl.Series(q_flow).alias("q"),
).sort("node_id").with_columns(
    pl.Series(list(net.res_bus["vm_pu"])).alias("v_pp")
).join(line_res_pp, on="node_id", how="left").with_columns(
    (c("v") - c("v_pp")).abs().alias("diff_v"),
    (c("p") - c("p_pp")).abs().alias("diff_p"),
    (c("q") - c("q_pp")).abs().alias("diff_q")
)

result["diff_v", "diff_p", "diff_q"].max()

diff_v,diff_p,diff_q
f64,f64,f64
0.103597,2.572768,0.748328
